In [10]:
from csv import DictReader
from pprint import PrettyPrinter
from rdflib import Graph, Literal, URIRef, RDF, RDFS, SKOS, SDO
from rdflib.namespace import Namespace, NamespaceManager

In [11]:
class Metadata(dict):
    def __init__(self):
        super().__init__()
        self["title"] = str()
        self["date"] = str()
        self["defines"] = str()
        
    def extractMetadata(self, row):
        self["title"] = row["dct:title"]
        self["date"] = row["dct:date"]
        self["defines"] = row["defines"]
        for key in row.keys():
            parts = key.split(" ")
            if "prefix" == parts[0]:
                prefix = parts[1]
                uri = row[key]
                if prefix[-1] == ":":
                    prefix = prefix[:-1]
                if uri[0] == "<":
                    uri = uri[1:]
                if uri[-1] == ">":
                    uri = uri[:-1]
                prefixes[prefix] = uri


In [12]:
class PropertyDefinition(dict):
    """Class for property defintions as dicts."""
    def __init__(self) :
        super().__init__()
        self["curi"] = str()
        self["label"] = str()
        self["comment"] = str()
        self["domain"] = str()
        self["range"] = str()

    def extractClassProp(self, domainCURI, propCURI):
        self["curi"] = propCURI
        self["domain"] = domainCURI
        self["range"] = "rdf:Literal"
    
    def extractLinkProperties(self, row):
        p["curi"] = row["Text Area 1"]
        if row["Source Arrow"] == "None" :
            domainId = row["Line Source"]
            p["domain"] = classes[domainId]["curi"]
        elif row["Destination Arrow"] == "None" :
            domainId = row["Line Destination"]
            p["domain"] = classes[domainId]["curi"]
        if row["Destination Arrow"] == "Arrow" :
            rangeId = row["Line Destination"]
            p["range"] = classes[rangeId]["curi"]
        elif row["Source Arrow"] == "Arrow" :
            rangeId = row["Line Source"]
            p["range"] = classes[rangeId]["curi"]              
        

In [13]:
class ClassDefinition(dict):
    """Class for class definition as dicts"""
    def __init__(self) :
        super().__init__()
        self["curi"] = str()
        self["label"] = str()
        self["comment"] = str()
        self["isDefinedBy"] = str()
        self["scopeNote"] = str()
        self["subClassOf"] = str()

    def extractClassDef(self, row):
            curi = row["Text Area 1"]
            [prefix, name] = splitCuri(curi)
            self["label"] = row["rdfs:label"]
            self["comment"] = row["rdfs:comment"]
            self["subClassOf"] = row["rdfs:subclassof"]
            self["scopeNote"] = row["skos:scopenote"]
            self["curi"] = curi

    def unpackProperties(self, string):
        """Turn a string with properties separted by linebreaks into a list of properties."""
        plist = string.split('\n')
        return plist


In [14]:
def splitCuri(curi):
    """Split a compact uri into prefix and name."""
    parts = curi.split(":")
    if len(parts) == 2 :
        return(parts)
    else:
        msg = "CURI should have one colon."
        raise Exception(msg)
        
def curi2URIRef(curi, namespaces):
    [ns_id, name] = splitCuri(curi)
    uri = namespaces[ns_id] + name
    return URIRef(uri)


In [15]:
pp = PrettyPrinter(indent = 2)
diag_fname = "DESM_Model.csv"

In [16]:
classes = dict()
#properties = dict()
properties = list()
prefixes = {"rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#"}
with open(diag_fname, "r") as diag_file:
    csvReader = DictReader(diag_file)
    metadata = Metadata()
    for row in csvReader:
        if row["Name"] == "Page":
            metadata.extractMetadata(row)
        elif row["Name"] == "Class":
            c = ClassDefinition()
            c.extractClassDef(row)
            classes[row["Id"]] = c
            class_props = c.unpackProperties(row["Text Area 2"])
            index = 1
            for prop in class_props:
                p = PropertyDefinition()
                p.extractClassProp(c["curi"], prop)
                p_id = row["Id"] +"."+ str(index)
#                properties[p_id] = p
                properties.append(p)
                index += index
        elif row["Name"] == "Line":
            p = PropertyDefinition()
            p.extractLinkProperties(row)
#            properties[row["Id"]] = p
            properties.append(p)
        else:
            next
        
#print("\n\nMetadata\n========")
#pp.pprint(metadata)
#print("\n\nPrefixes\n========")
pp.pprint(prefixes)
#print("\n\nClasses\n========")
pp.pprint(classes)
#print("\n\nProperties\n========")
#pp.pprint(properties)

{ 'dct': 'http://purl.org/dc/terms/',
  'desm': 'https://github.com/t3-innovation-network/desm/tree/main/schemas/desmSchema/',
  'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'}
{ '6': { 'comment': 'Resource aggregating mappings of two or more Abstract '
                    'Class created by one or more Data Standards Organizations '
                    '(DSOs).',
         'curi': 'desm:AbstractClassSet',
         'isDefinedBy': '',
         'label': 'Abstract Class Set',
         'scopeNote': 'This class is used only where a project creates '
                      'mappings to more than one absbtract class that are '
                      'intended by their creators to be viewed as a set.',
         'subClassOf': ''},
  '7': { 'comment': 'Resource describing mappings by one or more Data '
                    'Standards Organizations (DSOs) to a single abstract '
                    'class.',
         'curi': 'desm:AbstractClassMapping',
         'isDefinedBy': '',
         'label

In [17]:
schema = Graph()

for prefix in prefixes.keys():
    ns_uri = URIRef(prefixes[prefix])
    ns = Namespace(ns_uri)
    schema.bind(prefix, ns)
    if "base" == prefix.lower():
        self.sg.base = ns_uri
schema.bind("skos", SKOS)
schema.bind("sdo", SDO)

In [18]:
for k in classes.keys():
    curi = classes[k]["curi"]
    [ns_id, name] = splitCuri(curi)
    c = curi2URIRef(curi, prefixes)
    schema.add((c, RDF.type, RDFS.Class))
    if ns_id == metadata["defines"][:-1]:
        definedBy = URIRef(prefixes[ns_id])
        schema.add((c, RDFS.isDefinedBy, definedBy))
        schema.add((c, RDFS.label, Literal(classes[k]["label"], "en")))
        schema.add((c, RDFS.comment, Literal(classes[k]["comment"], "en")))
        if classes[k]["scopeNote"] != "":
            schema.add((c, SKOS.scopeNote, Literal(classes[k]["scopeNote"], "en")))

for prop in properties:
    curi = prop["curi"]
    [ns_id, name] = splitCuri(curi)
    p = curi2URIRef(curi, prefixes)
    schema.add((p, RDF.type, RDF.Property))
    definedBy = URIRef(prefixes[ns_id])
    schema.add((p, RDFS.isDefinedBy, definedBy))
    if ns_id == metadata["defines"][:-1]:
        if prop["domain"] != "":
            schema.add((p, RDFS.domain, curi2URIRef(prop["domain"], prefixes)))
        if prop["range"] != "":
            schema.add((p, RDFS.range, curi2URIRef(prop["range"], prefixes)))
    else:
        if prop["domain"] != "":
            schema.add((p, URIRef("http://schema.org/domainIncludes"), curi2URIRef(prop["domain"], prefixes)))
        if prop["range"] != "":
            schema.add((p, URIRef("http://schema.org/rangeIncludes"), curi2URIRef(prop["range"], prefixes)))
        
        

    
print(schema.serialize())


@prefix dct: <http://purl.org/dc/terms/> .
@prefix desm: <https://github.com/t3-innovation-network/desm/tree/main/schemas/desmSchema/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sdo: <http://schema.org/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .

desm:AbstractClassMapping a rdfs:Class ;
    rdfs:label "Abstract Class Mapping"@en ;
    rdfs:comment "Resource describing mappings by one or more Data Standards Organizations (DSOs) to a single abstract class."@en ;
    rdfs:isDefinedBy desm: .

desm:AbstractClassSet a rdfs:Class ;
    rdfs:label "Abstract Class Set"@en ;
    rdfs:comment "Resource aggregating mappings of two or more Abstract Class created by one or more Data Standards Organizations (DSOs)."@en ;
    rdfs:isDefinedBy desm: ;
    skos:scopeNote "This class is used only where a project creates mappings to more than one absbtract class that are intended by their creators to be vie